In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install ipynb
%cd 'drive/MyDrive/MSc Thesis'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/MSc Thesis'
/content/drive/MyDrive/MSc Thesis


In [ ]:
import pandas as pd
from pandas.errors import SettingWithCopyWarning
import numpy as np
import ast
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import spacy
import warnings
from sklearn.feature_selection import chi2
from sklearn import pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn import manifold
import gensim
import gensim.downloader as gensim_api
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K
from xgboost import XGBClassifier
# from ipynb.fs.full.lookup_table import custom_lookup_table

from glob import glob
from collections import defaultdict
import copy
import random

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
pd.options.display.max_colwidth = 300

Import and unify all datasets:

In [ ]:
TMP_Actions = import_data(
            glob=glob,
            pd=pd,
            low_memory=False,
            file_name_start='data/TMP_Actions.',
            names=['id','snapnum','meas_action_datetime','meas_action_acct_code_concerned','meas_action_cust_code_concerned','meas_action_comment'],
            lineterminator='\n',
            sep='\t'
           )

In [ ]:
TMP_Actions.dropna(subset=['snapnum'], inplace=True)
TMP_Actions = TMP_Actions.astype({"snapnum": int})

We check if there are duplicate comments made for the same account in the same snapshot (excluding comments such as ######):

In [ ]:
# Group by three columns and find the size of each group
group_sizes = TMP_Actions.groupby(['meas_action_acct_code_concerned', 'snapnum', 'meas_action_comment']).size()

# Select rows where group size is greater than 1
result = TMP_Actions.loc[(TMP_Actions.set_index(['meas_action_acct_code_concerned', 'snapnum', 'meas_action_comment'])\
    .index.isin(group_sizes[group_sizes > 1].index)) & (~(TMP_Actions['meas_action_comment'].str.contains('##########',\
    na=False)))]

result.reset_index(drop=True, inplace=True)

result.sort_values(by=['meas_action_acct_code_concerned','snapnum'])


,id,snapnum,meas_action_datetime,meas_action_acct_code_concerned,meas_action_cust_code_concerned,meas_action_comment
1547453,231646664,47,2019-11-27,344984.0,393604.0,εντολη διενεργειασ εαπ
1547454,231632140,47,2019-11-27,344984.0,373503.0,εντολη διενεργειασ εαπ
1546981,231646665,47,2019-11-27,344986.0,345338.0,εντολη διενεργειασ εαπ
1546982,231632141,47,2019-11-27,344986.0,345338.0,εντολη διενεργειασ εαπ
1481328,225837321,46,2019-10-04,344991.0,403750.0,### δα
...,...,...,...,...,...,...
1646820,230782994,47,2019-11-08,1122560.0,681951.0,[xxxx] δ.α. στο #######. το ####### ανενεργο
1646821,230782995,47,2019-11-08,1122560.0,681951.0,[xxxx] δ.α. στο #######. το ####### ανενεργο
1646822,230782997,47,2019-11-08,1122560.0,683068.0,[xxxx] δ.α.
1675457,237195158,48,2019-12-30,1122560.0,681951.0,####ε 30/9/#### [xxxx] (παραδοθηκε σε [xxxx] προσωπο)


It seems that we have accounts with more than one same comments in one month. This should be a point of consideration for our later analysis.

We continue with importing and preparing the Customers and Accounts datasets:

In [ ]:
DMCR_Unstruct_Customers = import_data(
            glob=glob,
            pd=pd,
            low_memory=False,
            file_name_start='data/DMCR_UNSTRUCT_CUSTOMERS.',
            names=['id','snapnum','meas_cusl_min_communication_date_3m'],
            lineterminator='\n',
            sep='	'
           )

In [ ]:
DMCR_Unstruct_Customers

,id,snapnum,meas_cusl_min_communication_date_3m
0,240628,37,2018-11-06\r
1,240629,37,2018-11-14\r
2,240716,37,\r
3,240630,37,2018-11-14\r
4,240631,37,2018-11-01\r
...,...,...,...
2368267,416887,48,\r
2368268,416888,48,2019-10-15\r
2368269,416889,48,2019-10-07\r
2368270,416890,48,2019-10-09\r


In [ ]:
DMCR_Unstruct_Customers['meas_cusl_min_communication_date_3m'] = \
    DMCR_Unstruct_Customers['meas_cusl_min_communication_date_3m'].str.replace("\r","")


In [ ]:
DMCR_Unstruct_Customers

,id,snapnum,meas_cusl_min_communication_date_3m
0,240628,37,2018-11-06
1,240629,37,2018-11-14
2,240716,37,
3,240630,37,2018-11-14
4,240631,37,2018-11-01
...,...,...,...
2368267,416887,48,
2368268,416888,48,2019-10-15
2368269,416889,48,2019-10-07
2368270,416890,48,2019-10-09


In [ ]:
DMCR_Unstruct_Accounts = import_data(
            glob=glob,
            pd=pd,
            low_memory=False,
            file_name_start='data/DMCR_UNSTRUCT_ACCOUNTS.',
            names=['id','snapnum','meas_acch_date_nominal','meas_acct_cust_code','meas_accl_appl_status',\
            'meas_accl_application_bucket','meas_accl_application_pending','meas_accl_paid_in_full_cm'],
            lineterminator='\n',
            sep='	'
           )

In [ ]:
DMCR_Unstruct_Accounts['meas_accl_paid_in_full_cm'] = \
    DMCR_Unstruct_Accounts['meas_accl_paid_in_full_cm'].str.replace("\r","")

In [ ]:
DMCR_Unstruct_Accounts

,id,snapnum,meas_acch_date_nominal,meas_acct_cust_code,meas_accl_appl_status,meas_accl_application_bucket,meas_accl_application_pending,meas_accl_paid_in_full_cm
0,346951,37,2019-01-01,303524,NaN,NaN,NaN,No
1,345151,37,2019-01-01,263870,NaN,NaN,NaN,No
2,345096,37,2019-01-01,323240,Review,NaN,Yes,No
3,346889,37,2019-01-01,319194,NaN,NaN,NaN,No
4,345656,37,2019-01-01,307450,NaN,NaN,NaN,No
...,...,...,...,...,...,...,...,...
4074749,1121760,48,2019-12-01,681743,NaN,NaN,NaN,No
4074750,1121761,48,2019-12-01,682781,NaN,NaN,NaN,No
4074751,1122348,48,2019-12-01,681873,NaN,NaN,NaN,No
4074752,1122349,48,2019-12-01,681370,NaN,NaN,NaN,No


Each account in DMCR_Unstruct_Accounts is related to only one customer:

In [ ]:
DMCR_Unstruct_Accounts.groupby('id')['meas_acct_cust_code'].nunique()\
    [DMCR_Unstruct_Accounts.groupby('id')['meas_acct_cust_code'].nunique()>1]

Series([], Name: meas_acct_cust_code, dtype: int64)

We have 3 datasets: TMP_Actions, DMCR_Unstruct_Customers and DMCR_Unstruct_Accounts

As a first step, we have to combine the data from DMCR_Unstruct_Customers and DMCR_Unstruct_Accounts and build the perimeter, in order to keep only the relevant data points. We are also going to construct the target for our dataset:

In [ ]:
# Filter based on conditions
a_filtered = DMCR_Unstruct_Accounts[
    (DMCR_Unstruct_Accounts["meas_accl_application_pending"].isnull() | (DMCR_Unstruct_Accounts["meas_accl_application_pending"] == "No")) &
    (DMCR_Unstruct_Accounts["meas_accl_appl_status"].isnull() |
     ~DMCR_Unstruct_Accounts["meas_accl_appl_status"].isin(['Approved', 'Running', 'Fulfilled', 'Partially Fulfilled', 'Out of Collection'])) &
    (DMCR_Unstruct_Accounts["snapnum"].between(37, 48))
].copy()

# Perform the merge operations
a_filtered["next_snapnum"]=a_filtered["snapnum"]+1
a_filtered["after_next_snapnum"]=a_filtered["snapnum"]+2
merged_a_b = pd.merge(a_filtered, DMCR_Unstruct_Customers, left_on=['meas_acct_cust_code', 'next_snapnum'], right_on=['id', 'snapnum'], suffixes=('', '_b'))

merged_a_c = pd.merge(merged_a_b, DMCR_Unstruct_Accounts, left_on=['id', 'next_snapnum'], right_on=['id', 'snapnum'], suffixes=('', '_c'))
merged_a_d = pd.merge(merged_a_c, DMCR_Unstruct_Accounts, left_on=['id', 'after_next_snapnum'], right_on=['id', 'snapnum'], suffixes=('', '_d'))

# Define a function to compute the labels
def define_target(row):
    c_value = row["meas_accl_paid_in_full_cm_c"] if pd.notna(row["meas_accl_paid_in_full_cm_c"]) else 'No'
    d_value = row["meas_accl_paid_in_full_cm_d"] if pd.notna(row["meas_accl_paid_in_full_cm_d"]) else 'No'

    return 1 if c_value == 'Yes' or d_value == 'Yes' else 0

merged_a_d['target'] = merged_a_d.apply(define_target, axis=1)

result = merged_a_d.groupby('target').size().reset_index(name='count')

print(result)

   target    count
0       0  3215957
1       1     4348


In [ ]:
Accounts_Customers_Final = merged_a_d[['id','meas_acct_cust_code','snapnum','meas_accl_paid_in_full_cm','meas_accl_appl_status',\
    'meas_accl_application_bucket','target']].copy()

Accounts_Customers_Final.rename(columns={'id': 'account_id', 'meas_acct_cust_code': 'customer_id'}, inplace=True)

The distribution of the classes:

In [ ]:
Accounts_Customers_Final

,account_id,customer_id,snapnum,meas_accl_paid_in_full_cm,meas_accl_appl_status,meas_accl_application_bucket,target
0,346951,303524,37,No,NaN,NaN,0
1,345151,263870,37,No,NaN,NaN,0
2,346889,319194,37,No,NaN,NaN,0
3,345656,307450,37,No,NaN,NaN,0
4,349325,307450,37,No,NaN,NaN,0
...,...,...,...,...,...,...,...
3220300,1122034,681070,46,No,NaN,NaN,0
3220301,1122433,681077,46,No,NaN,NaN,0
3220302,1121843,681078,46,No,NaN,NaN,0
3220303,1122464,681248,46,No,NaN,NaN,0


In [ ]:
Accounts_Customers_Final['target'].value_counts(normalize=True).round(4)

target
0    0.9986
1    0.0014
Name: proportion, dtype: float64

To create our dataset, we have to join Accounts_Customers_Final with TMP_Actions df which contains the comments:

In [ ]:
TMP_Actions.loc[TMP_Actions['meas_action_comment']=='NaN']

,id,snapnum,meas_action_datetime,meas_action_acct_code_concerned,meas_action_cust_code_concerned,meas_action_comment


In [ ]:
TMP_Actions['meas_action_datetime']=pd.to_datetime(TMP_Actions['meas_action_datetime'])
TMP_Actions['meas_action_comment_str']=TMP_Actions['meas_action_comment'].astype(str)


def merge_concat_comments(snap_history=-3):


    final_dataset = Accounts_Customers_Final.copy()

    for i in range(0, snap_history, -1):
        final_dataset[f'snapnum_{i}'] = final_dataset['snapnum'] + i

        final_dataset = final_dataset.merge(TMP_Actions, how='left', left_on=['account_id',f'snapnum_{i}'],\
            right_on=['meas_action_acct_code_concerned','snapnum'], suffixes=('', '_y'))

        final_dataset['meas_action_comment_str'].fillna('', inplace=True)
        final_dataset.loc[final_dataset['meas_action_comment_str'].str.lower() == 'nan', 'meas_action_comment_str'] = ''

        # final_dataset['meas_action_comment_concat'] = final_dataset.groupby(['account_id','snapnum'])['meas_action_comment_str']\
        #     .transform(lambda x:  'SAMESNAP ' + ' XXBOCOMMENT '.join(x))

        final_dataset[f'meas_action_comment_concat_{i}'] = final_dataset.sort_values(['meas_action_datetime'], ascending=False).\
            groupby(['account_id','snapnum'], as_index=False)['meas_action_comment_str'].transform(lambda x: ' '.join(x))

        final_dataset.drop(columns=['id', 'snapnum_y', f'snapnum_{i}', 'meas_action_datetime', 'meas_action_acct_code_concerned', 'meas_action_cust_code_concerned',\
            'meas_action_comment', 'meas_action_comment_str'], inplace=True)

        final_dataset.drop_duplicates(inplace=True)

        final_dataset = final_dataset.reset_index(drop=True)

        if i != 0:
            DMCR_Unstruct_Accounts[f'snapnum_{i}'] = DMCR_Unstruct_Accounts['snapnum']-i

            final_dataset = final_dataset.merge(DMCR_Unstruct_Accounts, how='left', left_on=['account_id','snapnum'], \
                right_on=['id',f'snapnum_{i}'], suffixes=('','_y'))

            final_dataset.drop(columns=['id', f'snapnum_{i}', 'snapnum_y', 'meas_acch_date_nominal', 'meas_acct_cust_code',\
               'meas_accl_appl_status_y', 'meas_accl_application_pending', 'meas_accl_paid_in_full_cm_y'], inplace=True)

            DMCR_Unstruct_Accounts.drop(columns=[f'snapnum_{i}'], inplace=True)

            final_dataset.rename(columns={'meas_accl_application_bucket_y': f'meas_accl_application_bucket_prev_{i}'}, inplace=True)

    return final_dataset

final_dataset = merge_concat_comments()


In [ ]:
final_dataset = final_dataset[['account_id', 'snapnum', 'meas_accl_paid_in_full_cm',\
       'meas_accl_appl_status', 'meas_accl_application_bucket', 'meas_accl_application_bucket_prev_-1',\
       'meas_accl_application_bucket_prev_-2','meas_action_comment_concat_0', 'meas_action_comment_concat_-1',\
       'meas_action_comment_concat_-2', 'target']]

In [ ]:
final_dataset = final_dataset.loc[((final_dataset['meas_action_comment_concat_0'] != '') & (final_dataset['meas_action_comment_concat_0'] != 'nan')) | \
              ((final_dataset['meas_action_comment_concat_-1'] != '') & (final_dataset['meas_action_comment_concat_-1'] != 'nan')) | \
              ((final_dataset['meas_action_comment_concat_-2'] != '') & (final_dataset['meas_action_comment_concat_-2'] != 'nan'))]

In [ ]:
final_dataset['target'].value_counts(normalize=True).round(4)

target
0    0.9982
1    0.0018
Name: proportion, dtype: float64

In [ ]:
# final_dataset.to_csv('final_dataset.csv', index=False)

In [ ]:
# final_dataset = pd.read_csv('/content/drive/MyDrive/MSc Thesis/final_dataset.csv')

<ipython-input-4-55985c8f2145>:1: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  final_dataset = pd.read_csv('/content/drive/MyDrive/MSc Thesis/final_dataset.csv')


In [ ]:
# final_dataset_sample = final_dataset.sample(10000).to_csv('/content/drive/MyDrive/MSc Thesis/final_dataset_sample.csv')